In [159]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import time
from geopy.geocoders import Nominatim

from geopy.geocoders import Nominatim


from geopy.extra.rate_limiter import RateLimiter


In [160]:
geolocator = Nominatim(user_agent="MSU CSE 404 Project Application")


def reverse_geocode(lat, lon):
    time.sleep(1.1) # to avoid rate limit

    return geolocator.reverse((lat, lon), exactly_one=True, language='en')


def create_grid(step=1.0):
    print("Creating grid...")
    lat_min = 24.396308
    lat_max = 49.384358
    lon_min = -125.0
    lon_max = -66.93457

    # Step size for the grid (degrees)
    latitudes = np.arange(lat_min, lat_max + step, step)
    longitudes = np.arange(lon_min, lon_max + step, step)

    print("total points in grid: " + str(len(latitudes) * len(longitudes)))
    print('Estimated time to complete:' + str(len(latitudes) * len(longitudes) * 1.1) + ' seconds')

    grid_points = [reverse_geocode(lat, lon) for lat in latitudes for lon in longitudes]
    grid_points = [location.raw for location in grid_points if location is not None]
    grid_points = [location for location in grid_points if location['address']['country_code'] == 'us']

    return grid_points



In [ ]:
SAVED_POINTS_PATH = "saved_points.csv"
REGENERATE_POINTS = False

df = None
if os.path.exists(SAVED_POINTS_PATH) and not REGENERATE_POINTS:
    df = pd.read_csv(SAVED_POINTS_PATH)
else:
    grid_points = create_grid(step=0.3)
    df = pd.DataFrame(grid_points)
    df.to_csv(SAVED_POINTS_PATH, index=False)

Creating grid...
total points in grid: 16575
Estimated time to complete:18232.5 seconds


In [162]:
# Convert lat and lon columns to numeric
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

fig = px.scatter_map(df, lat="lat", lon="lon", title="Grid Points in the Contiguous United States")
fig.show()

In [163]:
import requests
import random
from dotenv import load_dotenv

# Set up your API key and parameters
load_dotenv()
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
if GOOGLE_MAPS_API_KEY is None:
    raise ValueError("No API key found. Please set GOOGLE_MAPS_API_KEY in your .env file.")

size = '640x640'  # This is the maximum allowed size
url = 'https://maps.googleapis.com/maps/api/streetview'
key = GOOGLE_MAPS_API_KEY
source = 'outdoor'
radius = 50000 # The API will find the closest streetview, so just set a large radius to make sure we always get an image ( don't wanna waste our free quota ;) )

SAVE_PATH = "us_images"

for (i, row) in df.iterrows():
    filename = SAVE_PATH + os.path.sep + str(i) + '.jpg'
    image_already_exists = os.path.exists(filename)

    if image_already_exists and not REGENERATE_POINTS:
        print("Image already exists, skipping...")
        continue

    location = f"{row['lat']},{row['lon']}"

    heading = random.randint(0, 360)  # Random heading

    params = {
        'size': size,      
        'location': location,  
        'heading': heading,    
        'key': key,
        'source': source,
        'radius': radius,
        'return_error_code': True  
    }

    # Make the request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        
        # Save the image if successful
        with open(filename, 'wb') as file:
            file.write(response.content)
        print("Image saved as", filename)
    else:
        print(f"Error: {response.status_code}, {response.text}")

Image saved as us_images\0.jpg
Image saved as us_images\1.jpg
Image saved as us_images\2.jpg
Image saved as us_images\3.jpg
Image saved as us_images\4.jpg
Image saved as us_images\5.jpg
Image saved as us_images\6.jpg
Image saved as us_images\7.jpg
Image saved as us_images\8.jpg
Image saved as us_images\9.jpg
Image saved as us_images\10.jpg
Image saved as us_images\11.jpg
Image saved as us_images\12.jpg
Image saved as us_images\13.jpg
Image saved as us_images\14.jpg
Image saved as us_images\15.jpg
Image saved as us_images\16.jpg
Image saved as us_images\17.jpg
Image saved as us_images\18.jpg
Image saved as us_images\19.jpg
Image saved as us_images\20.jpg
Image saved as us_images\21.jpg
Image saved as us_images\22.jpg
Image saved as us_images\23.jpg
Image saved as us_images\24.jpg
Image saved as us_images\25.jpg
Image saved as us_images\26.jpg
Image saved as us_images\27.jpg
Image saved as us_images\28.jpg
Image saved as us_images\29.jpg
Image saved as us_images\30.jpg
Image saved as us_

KeyboardInterrupt: 